In [1]:
# Standard library imports
import math
import logging
from collections.abc import Mapping
from operator import attrgetter
from functools import partial
from copy import copy
from contextlib import contextmanager

# PyTorch imports
import torch
import torch.nn.functional as F
from torch import nn, tensor, optim
from torchvision import datasets
from torch.utils.data import DataLoader

# Visualization imports
import matplotlib.pyplot as plt
import matplotlib as mpl

# Data handling and datasets
from datasets import load_dataset, load_dataset_builder
import torchvision.transforms.functional as TF
from fastprogress import progress_bar, master_bar

# Fastcore utilities
import fastcore.all as fc
from fastcore.test import test_close

# Custom module imports
from miniai.conv import *
from miniai.datasets import *

In [2]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [3]:
logging.disable(logging.WARNING)

## DataLoaders

In [4]:
# Custom transform to flatten the input tensor
def flatten_transform(image):
    return torch.flatten(TF.to_tensor(image))

# Load the FashionMNIST dataset with the custom transform
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=flatten_transform  # Applying transform here
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=flatten_transform  # Applying transform here
)

# Create the train and test dataloaders
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

# Define a DataLoaders class to wrap the train and test dataloaders
class DataLoaders:
    def __init__(self, train, valid):
        self.train = train
        self.valid = valid

# Create the `dls` object with train and valid attributes
dls = DataLoaders(train_dataloader, test_dataloader)

In [5]:
dt = dls.train
xb,yb = next(iter(dt))
xb.shape, yb[:10]

(torch.Size([64, 784]), tensor([5, 4, 9, 4, 3, 0, 6, 5, 7, 6]))

## Learner

In [6]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD): 
        fc.store_attr()

    def one_batch(self):
        self.xb,self.yb = to_device(self.batch)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        with torch.no_grad(): 
            self.calc_stats()

    def calc_stats(self):
        acc = (self.preds.argmax(dim=1)==self.yb).float().sum()
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss*n)
        self.ns.append(n)

    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for self.num,self.batch in enumerate(dl): 
            self.one_batch()
        n = sum(self.ns)
        print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)
    
    def fit(self, n_epochs):
        self.accs,self.losses,self.ns = [],[],[]
        self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            with torch.no_grad(): self.one_epoch(False)

In [7]:
m,nh = 28*28,50
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10))

In [8]:
learn = Learner(model, dls, F.cross_entropy, lr=0.2)
learn.fit(1)

0 True 0.59319921875 0.7838333333333334
0 False 0.5811694754464286 0.7886571428571428


## Learner Callbacks

## Metrics

## Final Learner